##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Trabalhando com ClientData do TFF

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/working_with_client_data"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/federated/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

A noção de um dataset com clientes (por exemplo, usuários) como chave é essencial para a computação federada conforme modelada no TFF. O TFF conta com a interface [`tff.simulation.datasets.ClientData`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData) para fazer a abstração desse conceito, e todos os datasets que o TFF armazena ([stackoverflow](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/stackoverflow), [shakespeare](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/shakespeare), [emnist](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/emnist), [cifar100](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/cifar100) e [gldv2](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/gldv2)) implementam essa interface.

Se você estiver trabalhando com aprendizado federado usando seu próprio dataset, o TFF recomenda que você implemente a interface `ClientData` ou use uma das funções helpers do TFF para gerar um `ClientData` que represente seus dados no disco, como [`tff.simulation.datasets.ClientData.from_clients_and_fn`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData#from_clients_and_fn).

Como a maioria dos exemplos completos do TFF começam com objetos `ClientData`, implementar a interface `ClientData` com seu próprio dataset personalizado facilitará o uso de código existente escrito com o TFF. Além disso, `tf.data.Datasets` construídos por `ClientData` podem ser iterados diretamente para gerar estruturas de arrays `numpy`, então objetos `ClientData` podem ser usados com qualquer framework de aprendizado de máquina baseado em Python antes de migrar para o TFF.

Existem diversos padrões que você pode usar para facilitar sua vida se deseja escalonar suas simulações para diversas máquinas ou implantá-las. Falaremos abaixo sobre algumas maneiras de usar `ClientData` e TFF para facilitar o máximo possível sair de desenvolvimento para produção e de pequena escala para larga escala.

## Que padrão devo usar para passar ClientData ao TFF?

Vamos falar sobre dois usos de `ClientData` do TFF com profundidade. Se você se encaixar em uma das duas categorias abaixo, vai preferir claramente uma delas. Caso contrário, pode precisar entender melhor as vantagens e desvantagens de cada uma para tomar uma decisão mais fundamentada.

- Quero iterar o mais rápido possível em uma máquina local; não preciso aproveitar o runtime distribuído do TFF.

    - Você vai querer passar `tf.data.Datasets` ao TFF diretamente.
    - Isso permite que você programe de forma imperativa usando objetos `tf.data.Dataset` e processe-os de forma arbitrária.
    - Isso proporciona maior flexibilidade do que a opção abaixo; enviar a lógica aos clientes requer que ela seja serializável.

- Quero executar minha computação federada no runtime remoto do TFF ou planejo fazê-lo em breve.

    - Neste caso, você vai querer mapear a construção do dataset e o pré-processamento para os clientes.
    - Isso implica que você deve simplesmente passar uma lista de `client_ids` diretamente para sua computação federada.

- Enviar a construção e o pré-processamento do dataset para os clientes evita gargalos na serialização e aumenta consideravelmente o desempenho com centenas ou milhares de clientes.

In [ ]:
#@title Set up open-source environment
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow_federated

In [1]:
#@title Import packages
import collections
import time

import tensorflow as tf
import tensorflow_federated as tff

## Manipulação de um objeto ClientData

Vamos começar carregando e explorando o `ClientData` EMNIST do TFF:


In [2]:
client_data, _ = tff.simulation.datasets.emnist.load_data()

Ao inspecionar o primeiro dataset, podemos ver que tipos de exemplos estão em `ClientData`.

In [3]:
first_client_id = client_data.client_ids[0]
first_client_dataset = client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)
# This information is also available as a `ClientData` property:
assert client_data.element_type_structure == first_client_dataset.element_spec

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])


Observe que o dataset gera objetos `collections.OrderedDict` que têm chaves `pixels` e `label`, em que pixels é um tensor com formato `[28, 28]`. Suponha que queiramos achatar as entradas para o formato `[784]`. Uma possível forma de fazer isso é aplicar uma função de pré-processamento ao objeto `ClientData`.

In [4]:
def preprocess_dataset(dataset):
  """Create batches of 5 examples, and limit to 3 batches."""

  def map_fn(input):
    return collections.OrderedDict(
        x=tf.reshape(input['pixels'], shape=(-1, 784)),
        y=tf.cast(tf.reshape(input['label'], shape=(-1, 1)), tf.int64),
    )

  return dataset.batch(5).map(
      map_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE).take(5)


preprocessed_client_data = client_data.preprocess(preprocess_dataset)

# Notice that we have both reshaped and renamed the elements of the ordered dict.
first_client_dataset = preprocessed_client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


Além disso, poderemos querer fazer algum pré-processamento mais complexo (e possivelmente stateful), como misturar os dados.

In [5]:
def preprocess_and_shuffle(dataset):
  """Applies `preprocess_dataset` above and shuffles the result."""
  preprocessed = preprocess_dataset(dataset)
  return preprocessed.shuffle(buffer_size=5)

preprocessed_and_shuffled = client_data.preprocess(preprocess_and_shuffle)

# The type signature will remain the same, but the batches will be shuffled.
first_client_dataset = preprocessed_and_shuffled.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


## Fazendo interface com `tff.Computation`

Agora que podemos fazer algumas manipulações básicas com objetos `ClientData`, estamos prontos para alimentar dados a uma `tff.Computation`. Definimos um [`tff.templates.IterativeProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess), que implementa o [cálculo federado de médias](https://arxiv.org/abs/1602.05629), e exploramos diferentes métodos de passar os dados a ele.

In [6]:
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
  ])
  return tff.learning.models.from_keras_model(
      model,
      # Note: input spec is the _batched_ shape, and includes the 
      # label tensor which will be passed to the loss function. This model is
      # therefore configured to accept data _after_ it has been preprocessed.
      input_spec=collections.OrderedDict(
          x=tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
          y=tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  
trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01))

Antes de começarmos a trabalhar com esse `IterativeProcess`, vale a pena tecer um comentário sobre a semântica de `ClientData`. Um objeto `ClientData` representa a *totalidade* da população disponível para treinamento federado, que, em geral, [não está disponível para o ambiente de execução de um sistema de aprendizado federado em produção](https://arxiv.org/abs/1902.01046) e é específico à simulação. De fato, `ClientData` dá ao usuário a capacidade de contornar totalmente a computação federada e apenas treinar um modelo no lado do servidor como sempre via [`ClientData.create_tf_dataset_from_all_clients`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData?hl=en#create_tf_dataset_from_all_clients).

O ambiente de simulação do TFF dá controle total do loop externo ao pesquisador. Especificamente, isso traz considerações de disponibilidade de clientes, dropout de clientes, etc., que precisam ser tratados pelo usuário ou pelo driver script do Python. Por exemplo, seria possível modelar o dropout de clientes ajustando a distribuição da amostragem para os `client_ids` de `ClientData` de tal forma que usuários com mais dados (e, consequentemente, computações locais com execução mais longa) seriam selecionados com probabilidade mais baixa.

Porém, em um sistema federado real, os clientes não podem ser selecionados explicitamente pelo trainer do modelo. A seleção de clientes é delegada ao sistema que está executando a computação federada.

### Passando `tf.data.Datasets` diretamente para o TFF

Uma opção que temos para fazer a interface entre um `ClientData` e um `IterativeProcess` é construir `tf.data.Datasets` no Python e passá-los ao TFF.

Observe que, se usarmos nosso `ClientData` pré-processado, os datasets gerados terão o tipo apropriado esperado pelo modelo definido acima.

In [7]:
selected_client_ids = preprocessed_and_shuffled.client_ids[:10]

preprocessed_data_for_clients = [
    preprocessed_and_shuffled.create_tf_dataset_for_client(
        selected_client_ids[i]) for i in range(10)
]

state = trainer.initialize()
for _ in range(5):
  t1 = time.time()
  result = trainer.next(state, preprocessed_data_for_clients)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.934802532196045, round time 2.5420753955841064
loss 3.350963830947876, round time 0.45527172088623047
loss 3.1382687091827393, round time 0.47087883949279785
loss 3.0774152278900146, round time 0.4089682102203369
loss 2.9193594455718994, round time 0.3964221477508545


Porém, se formos por esse caminho, ***não conseguiremos mudar de forma trivial para simulação com várias máquinas***. Os datasets que construímos no runtime local do TensorFlow podem *capturar o estado do ambiente Python circundante*, e pode haver falha na serialização ou desserialização quando eles tentarem referenciar o estado que não está mais disponível para eles. Por exemplo, isso pode se manifestar no erro irrecuperável do `tensor_util.cc` do TensorFlow:

```
Check failed: DT_VARIANT == input.dtype() (21 vs. 20)
```

### Mapeamento da construção e do pré-processamento para os clientes

Para evitar esse problema, o TFF recomenda aos usuários que considerem o instanciamento e pré-processamento de datasets como *algo que ocorre localmente em cada cliente* e que usem os helpers do TFF ou `federated_map` para executar explicitamente esse código de pré-processamento em cada cliente.

Conceitualmente, o motivo para preferir isso é claro: no runtime local do TFF, os clientes têm acesso ao ambiente global do Python apenas "acidentalmente" devido ao fato de toda a orquestração federada ocorrer em uma única máquina. Vale a pena salientar que um raciocínio similar leva à filosofia funcional, entre plataformas e sempre serializável do TFF.

O TFF torna essa mudança simples através do atributo de `ClientData` `dataset_computation`, uma `tff.Computation` que recebe um `client_id` e retorna o `tf.data.Dataset` associado.

Observe que `preprocess` funciona com `dataset_computation`; o atributo `dataset_computation` do `ClientData` pré-processado incorpora todo o pipeline de pré-processamento que acabamos de definir:

In [8]:
print('dataset computation without preprocessing:')
print(client_data.dataset_computation.type_signature)
print('\n')
print('dataset computation with preprocessing:')
print(preprocessed_and_shuffled.dataset_computation.type_signature)

dataset computation without preprocessing:
(string -> <label=int32,pixels=float32[28,28]>*)


dataset computation with preprocessing:
(string -> <x=float32[?,784],y=int64[?,1]>*)


Poderíamos invocar `dataset_computation` e receber um dataset eager no runtime do Python, mas o verdadeiro poder dessa estratégia é quando combinamos com um processo iterativo ou outra computação para evitar a materialização desses datasets no runtime global eager. O TFF conta com uma função helper [`tff.simulation.compose_dataset_computation_with_iterative_process`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/compose_dataset_computation_with_iterative_process), que pode ser usada para fazer exatamente isso.

In [9]:
trainer_accepting_ids = tff.simulation.compose_dataset_computation_with_iterative_process(
    preprocessed_and_shuffled.dataset_computation, trainer)

Tanto esses `tff.templates.IterativeProcesses` quanto o acima executam da mesma forma, porém, o primeiro aceita datasets de cliente pré-processados, e o segundo aceita strings representando IDs de cliente, tratando tanto a construção quanto o pré-processamento do dataset em seu corpo. De fato, `state` pode ser passado entre os dois.

In [10]:
for _ in range(5):
  t1 = time.time()
  result = trainer_accepting_ids.next(state, selected_client_ids)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.6114611625671387, round time 1.4935951232910156
loss 2.612247943878174, round time 0.30751872062683105
loss 2.8368589878082275, round time 0.3043978214263916
loss 2.6863903999328613, round time 0.3107311725616455
loss 2.6816341876983643, round time 0.4325370788574219


### Dimensionando para um grande número de clientes

`trainer_accepting_ids` pode ser usado imediatamente no runtime multimáquina do TFF e evita materializar os `tf.data.Datasets` e o controlador (e, portanto, serializa-os e envia-os para os workers).

Isso acelera consideravelmente as simulações distribuídas, especialmente com um grande número de clientes, e também permite agregação intermediária para evitar sobrecarga de serialização/desserialização similar.


### Detalhe opcional: combinação manual de lógica de pré-processamento no TFF

O TFF foi criado para permitir combinações do zero. O tipo de combinação realizado pelo helper do TFF também está sob nosso controle como usuários. Poderíamos combinar manualmente a computação de pré-processamento que acabamos de definir com o `next` do trainer de maneira bem simples:

In [11]:
selected_clients_type = tff.FederatedType(preprocessed_and_shuffled.dataset_computation.type_signature.parameter, tff.CLIENTS)

@tff.federated_computation(trainer.next.type_signature.parameter[0], selected_clients_type)
def new_next(server_state, selected_clients):
  preprocessed_data = tff.federated_map(preprocessed_and_shuffled.dataset_computation, selected_clients)
  return trainer.next(server_state, preprocessed_data)

manual_trainer_with_preprocessing = tff.templates.IterativeProcess(initialize_fn=trainer.initialize, next_fn=new_next)

De fato, isso é exatamente o que o helper que usamos está fazendo nos bastidores (além da checagem e manipulação de tipos adequadas). Poderíamos até mesmo ter expressado a mesma lógica de maneira ligeiramente diferente por meio da serialização de `preprocess_and_shuffle` em uma `tff.Computation` e por meio da decomposição de `federated_map` em um passo que constrói datasets não processados e em outro que executa `preprocess_and_shuffle` em cada cliente.

Podemos verificar que esse caminho mais manual resulta em computações com a mesma assinatura de tipos que do helper do TFF (nomes do parâmetro do módulo):

In [12]:
print(trainer_accepting_ids.next.type_signature)
print(manual_trainer_with_preprocessing.next.type_signature)

(<state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,client_data={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,metrics=<distributor=<>,client_work=<train=<sparse_categorical_accuracy=float32,loss=float32,num_examples=int64,num_batches=int64>>,aggregator=<mean_value=<>,mean_weight=<>>,finalizer=<>>@SERVER>)
(<server_state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,selected_clients={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggre